In [ ]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'Tesla T4')

In [ ]:
!pip install -U transformers accelerate bitsandbytes

In [ ]:
!pip install -U sentencepiece

In [ ]:
import json
import hashlib
from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
INPUT_DIR = Path("/content/data_unificada")
OUTPUT_DIR = Path("/content/data_skills_final")
INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

print("✅ Modelo cargado correctamente")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Modelo cargado correctamente


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def gen_id(prefix, text):
    base = f"{prefix}_{text}".encode("utf-8")
    return f"{prefix}_{hashlib.sha1(base).hexdigest()[:10]}"

def safe(value, default=""):
    return value if value is not None else default

def limpiar_skill(texto):
    return texto.strip().lower()

In [ ]:
def generar_prompt_carrera(career, materias_bloque):
    if len(materias_bloque) <= 2:
        materias_texto = "\n".join(f"- {m}" for m in materias_bloque)
        return f"""<|begin_of_text|>
    <|system|>
    Eres un orientador vocacional universitario experto en Ecuador.
    Tu objetivo es ayudar a estudiantes de último año de colegio a decidir si una carrera es adecuada para ellos.
    Respondes únicamente en JSON válido, sin texto adicional.
    <|end|>

    <|user|>
    Genera habilidades, intereses y tipos de actividades que una persona desarrollará
    y realizará frecuentemente al estudiar la siguiente carrera universitaria.

    IMPORTANTE:
    Estas habilidades deben servir para que un estudiante de colegio pueda decidir
    si esta carrera encaja con su personalidad, gustos e intereses,
    NO es un perfil de egreso académico.

    Carrera: {career['career_name']}
    Universidad: {career['university_name']}

    Este bloque incluye solo parte de las materias de la carrera:
    Materias del bloque:
    {materias_texto}

    Descripción institucional resumida:
    {career.get('description', '')[:500]}

    Reglas:
    - Idioma: español
    - 1 habilidades o afinidades por bloque
    - Redacción clara y concreta
    - Enfocadas en lo que el estudiante HARÁ y LE GUSTARÁ
    - Evitar términos genéricos sin contexto
    - No usar placeholders
    - No repetir ideas
    - Responde SOLO un objeto JSON

    Formato exacto:
    {{
      "skills": [
          "habilidad 1",
          "habilidad 2"
      ]
    }}
    <|end|>

    <|assistant|>
    """
    elif len(materias_bloque) <= 5:

      materias_texto = "\n".join(f"- {m}" for m in materias_bloque)
      return f"""<|begin_of_text|>
  <|system|>
  Eres un orientador vocacional universitario experto en Ecuador.
  Tu objetivo es ayudar a estudiantes de último año de colegio a decidir si una carrera es adecuada para ellos.
  Respondes únicamente en JSON válido, sin texto adicional.
  <|end|>

  <|user|>
  Genera habilidades, intereses y tipos de actividades que una persona desarrollará
  y realizará frecuentemente al estudiar la siguiente carrera universitaria.

  IMPORTANTE:
  Estas habilidades deben servir para que un estudiante de colegio pueda decidir
  si esta carrera encaja con su personalidad, gustos e intereses,
  NO es un perfil de egreso académico.

  Carrera: {career['career_name']}
  Universidad: {career['university_name']}

  Este bloque incluye solo parte de las materias de la carrera:
  Materias del bloque:
  {materias_texto}

  Descripción institucional resumida:
  {career.get('description', '')[:500]}

  Reglas:
  - Idioma: español
  - 5 habilidades o afinidades por bloque
  - Redacción clara y concreta
  - Enfocadas en lo que el estudiante HARÁ y LE GUSTARÁ
  - Evitar términos genéricos sin contexto
  - No usar placeholders
  - No repetir ideas
  - Responde SOLO un objeto JSON

  Formato exacto:
  {{
    "skills": [
        "habilidad 1",
        "habilidad 2"
    ]
  }}
  <|end|>

  <|assistant|>
  """

    else:

      materias_texto = "\n".join(f"- {m}" for m in materias_bloque)
      return f"""<|begin_of_text|>
  <|system|>
  Eres un orientador vocacional universitario experto en Ecuador.
  Tu objetivo es ayudar a estudiantes de último año de colegio a decidir si una carrera es adecuada para ellos.
  Respondes únicamente en JSON válido, sin texto adicional.
  <|end|>

  <|user|>
  Genera habilidades, intereses y tipos de actividades que una persona desarrollará
  y realizará frecuentemente al estudiar la siguiente carrera universitaria.

  IMPORTANTE:
  Estas habilidades deben servir para que un estudiante de colegio pueda decidir
  si esta carrera encaja con su personalidad, gustos e intereses,
  NO es un perfil de egreso académico.

  Carrera: {career['career_name']}
  Universidad: {career['university_name']}

  Este bloque incluye solo parte de las materias de la carrera:
  Materias del bloque:
  {materias_texto}

  Descripción institucional resumida:
  {career.get('description', '')[:500]}

  Reglas:
  - Idioma: español
  - 10 habilidades o afinidades por bloque
  - Redacción clara y concreta
  - Enfocadas en lo que el estudiante HARÁ y LE GUSTARÁ
  - Evitar términos genéricos sin contexto
  - No usar placeholders
  - No repetir ideas
  - Responde SOLO un objeto JSON

  Formato exacto:
  {{
    "skills": [
        "habilidad 1",
        "habilidad 2"
    ]
  }}
  <|end|>

  <|assistant|>
  """


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def inferir_skills(prompt):
    inputs = tokenizer(
      prompt,
      return_tensors="pt",
      truncation=True,
      max_length=2048,
      padding=True
    )

    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        do_sample=False,
        temperature=0.2,
        repetition_penalty=1.15,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )

    texto = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(texto)

    # 🔥 EXTRAER SOLO EL PRIMER JSON
    if "<|assistant|>" in texto:
        texto = texto.split("<|assistant|>")[-1].strip()

    start = texto.find("{")
    end = texto.rfind("}") + 1


    if start == -1 or end == -1:
        return []

    json_text = texto[start:end]

    try:
        data = json.loads(json_text)
        return data.get("skills", [])
    except json.JSONDecodeError:
        return []

In [ ]:
def main():
    resultado_final = []

    BLOQUE_SIZE = 10  # número de materias por bloque, puedes ajustar
    EXCLUDE_KEYWORDS = [
        "titulación", "práctica", "vinculación", "optativa", "itinerario",
        "laboratorio", "electiva", "final", "complementario", "proyecto",
        "titulacion", "practica", "practicas", "prácticas", "vinculacion"
    ]

    for archivo in INPUT_DIR.glob("*.json"):
        print(f"\n📂 Procesando archivo: {archivo.name}")

        with open(archivo, "r", encoding="utf-8") as f:
            careers = json.load(f)

        for career in careers:
            career_name = safe(career.get("career_name"))
            career_id = safe(career.get("career_id"), gen_id("career", career_name))
            university = safe(career.get("university_name"))
            locations = career.get("locations", [])

            # filtrar materias según keywords
            materias = [
                safe(sub.get("name"))
                for sub in career.get("subjects", [])
                if safe(sub.get("name")) and not any(
                    kw.lower() in safe(sub.get("name")).lower() for kw in EXCLUDE_KEYWORDS
                )
            ]

            if not materias:
                print(f"⚠️ No hay materias normales en la carrera {career_name}, se omite")
                continue

            print(f"\n🎓 Carrera: {career_name}")
            print(f"📘 Materias normales encontradas: {len(materias)}")

            all_skills = set()
            subjects_out = []

            # dividir materias en bloques
            for i in range(0, len(materias), BLOQUE_SIZE):
                bloque = materias[i:i+BLOQUE_SIZE]
                print(f"⚡ Generando skills para bloque {i//BLOQUE_SIZE + 1} ({len(bloque)} materias)...")

                prompt = generar_prompt_carrera(career, bloque)
                skills_bloque = inferir_skills(prompt)

                print(f"    - {len(skills_bloque)} skills encontradas en el bloque")

                for sk in skills_bloque:
                    sk_clean = limpiar_skill(sk)
                    all_skills.add(sk_clean)

            if not all_skills:
                print(f"⚠️ No se generaron skills para la carrera {career_name}, se omite")
                continue

            # crear subjects simplificados: solo código, nombre y semestre
            for sub in career.get("subjects", []):
                subject_name = safe(sub.get("name"))
                if not subject_name or any(kw.lower() in subject_name.lower() for kw in EXCLUDE_KEYWORDS):
                    continue

                subjects_out.append({
                    "code": safe(sub.get("code")),
                    "name": subject_name,
                    "semester": safe(sub.get("semester"))
                })

            resultado_final.append({
                "career_id": career_id,
                "career_name": career_name,
                "university_name": university,
                "country": "ECUADOR",
                "locations": locations,
                "subjects": subjects_out,
                "career_general_skills": sorted(all_skills)  # solo strings, unidas
            })

            print(f"✅ Skills totales de la carrera: {len(all_skills)}")

    output_file = OUTPUT_DIR / "careers_skills_ecuador.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(resultado_final, f, indent=2, ensure_ascii=False)

    print(f"\n🚀 Proceso terminado → {output_file}")


In [ ]:
if __name__ == "__main__":
    main()

Se han truncado las últimas 5000 líneas del flujo de salida.
  - No repetir ideas
  - Responde SOLO un objeto JSON

  Formato exacto:
  {
    "skills": [
        "habilidad 1",
        "habilidad 2"
    ]
  }
  <|end|>

  <|assistant|>
   {
     "skills": [
       "Diseñar y crear espacios funcionales y estéticos",
       "Desarrollar habilidades técnicas y creativas en diseño y construcción",
       "Comunicarse efectivamente con clientes y equipo de trabajo",
       "Analizar y resolver problemas de espacio y ergonomía",
       "Utilizar software de diseño asistido por computadora (CAD) y otros herramientas digitales",
       "Elegir materiales y productos para proyectos de diseño interior",
       "Considerar aspectos ambientales y sostenibles en el diseño",
       "Trabajar en equipo y colaborativamente",
       "Presentar y explicar proyectos de diseño interior",
       "Investigar y analizar tendencias y tendencias en diseño interior"
     ]
   }
    - 10 skills encontradas en el